In [3]:
!apt install swig
!pip install orthnet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 2s (563 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../swig3.0_

In [4]:

import torch
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
from torch import randn
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn

import random, os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd

from orthnet import Legendre, Chebyshev

%matplotlib inline

In [17]:
cuda0 = torch.device('cuda:0')
print(torch.cuda.device_count(), torch.cuda.get_device_name(0))

1 Tesla K80


In [34]:
torch.cuda.init()

In [18]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [19]:
class LegendreActivation(nn.Module):
    def __init__(self,degree):
        super().__init__()
        self.degree = degree
        self.data = None
        
        self.D = torch.zeros((degree , degree )) 
        for i in range(degree):
          for j in range(0 , i):
            if (i + j) % 2 == 1:
              self.D[i, j] = 2 * (j + 1) - 1
    
    def forward(self, X):              
      data = Legendre(X, self.degree).tensor
      self.data = data
      return data

    def backward(self,):
      return (self.D @ (self.data).T).T


class LegendreBlock(nn.Module):
    def __init__(self, n_input, degree):        
        super().__init__()
        self.degree = degree - 1
        self.n_input = n_input
        self.linear = nn.Linear(self.n_input, 1).double()
        self.tanh = nn.Tanh().double()
        self.Legendre = LegendreActivation(self.degree)

    def forward(self, X):      
      X = self.tanh(self.linear(X))      
      data = self.Legendre(X)
      return data

class ChebyshevActivation(nn.Module):
    def __init__(self,degree):
        super().__init__()
        self.degree = degree
        self.data = None
        
        self.D = torch.zeros((degree , degree )) 
        for i in range(degree):
          for j in range(0 ,i):
            if (i+j) % 2 == 1:
              self.D[i, j] = 2 * i
              if j == 0:
                self.D[i, j] = self.D[i, j]/2.0
    def forward(self, X):              
      data = Chebyshev(X, self.degree).tensor
      self.data = data
      return data

    def backward(self,):
      return (self.D @ (self.data).T).T

class ChebyshevBlock(nn.Module):
    def __init__(self, n_input, degree):        
        super().__init__()
        self.degree = degree - 1
        
        self.n_input = n_input
        self.linear = nn.Linear(self.n_input, 1).double()
        self.tanh = nn.Tanh().double()
        self.Chebyshev = ChebyshevActivation(self.degree)


    def forward(self, X):      
      X = self.tanh(self.linear(X))      
      data = self.Chebyshev(X)
      
      return data

In [37]:
def dy_dx(y, x):
  return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), create_graph=True)[0]

def d2y_dx2(y, x):
  return dy_dx(dy_dx(y,x), x)

def d3y_dx3(y, x):
  return dy_dx(d2y_dx2(y,x), x)

In [21]:
domain = [0, 1]
n_discretization = 1000 * domain[1] - domain[0]

In [23]:
n_input = 1
n_output = 1
eps = 1e-10

In [25]:
x = torch.linspace(domain[0] + eps, domain[1] - eps, n_discretization, dtype=torch.double, device=cuda0).reshape(-1,1)
x = Variable(x, requires_grad=True).double()
print(x)

tensor([[1.0000e-10],
        [1.0010e-03],
        [2.0020e-03],
        [3.0030e-03],
        [4.0040e-03],
        [5.0050e-03],
        [6.0060e-03],
        [7.0070e-03],
        [8.0080e-03],
        [9.0090e-03],
        [1.0010e-02],
        [1.1011e-02],
        [1.2012e-02],
        [1.3013e-02],
        [1.4014e-02],
        [1.5015e-02],
        [1.6016e-02],
        [1.7017e-02],
        [1.8018e-02],
        [1.9019e-02],
        [2.0020e-02],
        [2.1021e-02],
        [2.2022e-02],
        [2.3023e-02],
        [2.4024e-02],
        [2.5025e-02],
        [2.6026e-02],
        [2.7027e-02],
        [2.8028e-02],
        [2.9029e-02],
        [3.0030e-02],
        [3.1031e-02],
        [3.2032e-02],
        [3.3033e-02],
        [3.4034e-02],
        [3.5035e-02],
        [3.6036e-02],
        [3.7037e-02],
        [3.8038e-02],
        [3.9039e-02],
        [4.0040e-02],
        [4.1041e-02],
        [4.2042e-02],
        [4.3043e-02],
        [4.4044e-02],
        [4

In [26]:
mlp = nn.Sequential(  
  LegendreBlock(n_input, 32),
  nn.Linear(32, 64),
  nn.Tanh(),
  nn.Linear(64, 128),
  nn.Tanh(),
  nn.Linear(128, 64),
  nn.Tanh(),
  nn.Linear(64, 16),
  nn.Linear(16, n_output)
).double()


pytorch_total_params = sum(p.numel() for p in mlp.parameters() if p.requires_grad)
pytorch_total_params

19747

In [35]:
mlp.cuda()

Sequential(
  (0): LegendreBlock(
    (linear): Linear(in_features=1, out_features=1, bias=True)
    (tanh): Tanh()
    (Legendre): LegendreActivation()
  )
  (1): Linear(in_features=32, out_features=64, bias=True)
  (2): Tanh()
  (3): Linear(in_features=64, out_features=128, bias=True)
  (4): Tanh()
  (5): Linear(in_features=128, out_features=64, bias=True)
  (6): Tanh()
  (7): Linear(in_features=64, out_features=16, bias=True)
  (8): Linear(in_features=16, out_features=1, bias=True)
)

In [27]:
def get_loss(x, ret_res=False):
  y = mlp(x)
  y_x = dy_dx(y, x)
  y_xx = dy_dx(y_x, x)
  y_xxx = dy_dx(y_xx, x)


  """    
  Blasius Eq.: 
   f''' + 0.5 ff'' = 0
   2f''' + ff'' = 0   ,   f(0) = f'(0) = 0, f'(∞) = 1
  """
  
  residual = (2 * y_xxx) + (y_xx * y)


  # boundaries same for all equations
  boundary1 = y[0]
  boundary2 = y_x[0]
  boundary3 = y_x[-1] - 1

  loss = (residual**2).mean() + boundary1**2 + boundary2**2 + boundary3**2
  return (loss, residual) if ret_res else loss

In [28]:
def closure():
  loss = get_loss(x)
  optimizer.zero_grad()
  loss.backward()
  return loss

In [42]:
#TODO
optimizer = optim.Adam(list(mlp.parameters()), lr=0.01, betas=(0.9, 0.999), eps=1e-32)
previous = 0
losses = []
epoch_Adam = 100
epoch_LBFGS = 5000
for i in range(epoch_Adam):
  loss = get_loss(x)
  
  if i % 2 == 0:        
    print('Step: %03d loss = %016.10f' % (i, loss))        
  
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  losses.append(loss.detach().cpu().numpy())


optimizer = optim.LBFGS(list(mlp.parameters()), lr = 0.01)
previous = 0
for i in range(epoch_LBFGS):
  loss = get_loss(x)
  if i % 2 == 0:        
    print('Step: %03d loss = %016.10f' % (i+epoch_Adam, loss))
    if abs(previous - loss) < 1e-10:
        print('converged')
        break
    
    previous = loss
  
  losses.append(loss.detach().cpu().numpy())
  optimizer.step(closure)

print("Final loss = %.2e" % get_loss(x))

Step: 000 loss = 00000.0000259118
Step: 002 loss = 00055.9813210930
Step: 004 loss = 00004.2331779141
Step: 006 loss = 00002.6394862677
Step: 008 loss = 00001.1261426759
Step: 010 loss = 00002.0153609354
Step: 012 loss = 00001.6004538459
Step: 014 loss = 00001.0001981281
Step: 016 loss = 00001.3124108169
Step: 018 loss = 00001.3887521732
Step: 020 loss = 00001.0714704606
Step: 022 loss = 00001.0373027992
Step: 024 loss = 00001.1824445381
Step: 026 loss = 00001.1051660574
Step: 028 loss = 00000.9901707526
Step: 030 loss = 00001.0308831079
Step: 032 loss = 00001.0467232540
Step: 034 loss = 00000.9663918486
Step: 036 loss = 00000.9613893124
Step: 038 loss = 00000.9608777758
Step: 040 loss = 00000.9079678382
Step: 042 loss = 00000.9054134658
Step: 044 loss = 00000.9092250277
Step: 046 loss = 00000.8967732128
Step: 048 loss = 00000.9554889584
Step: 050 loss = 00000.9253969717
Step: 052 loss = 00000.8060863930
Step: 054 loss = 00000.8899797971
Step: 056 loss = 00000.7796520173
Step: 058 loss

In [46]:
pd.options.display.float_format = '{:.16f}'.format
domain_test = torch.tensor([0.00,0.10,0.20,0.50,1.00,1.50,2.00], dtype=torch.double).reshape(-1,1)
predict_test = mlp.cpu().forward(domain_test).detach().numpy().flatten()
pd.DataFrame(np.array([domain_test.numpy().flatten(), predict_test]).T, columns=['x','y'])

,x,y
0,0.0000000000000000,-0.0000027895915439
1,0.1000000000000000,0.0051042379946116
2,0.2000000000000000,0.0204212138627372
3,0.5000000000000000,0.1275088286981601
4,1.0000000000000000,0.5062977524877190
5,1.5000000000000000,1.1099985865726627
6,2.0000000000000000,1.6378331452849346


In [48]:
# calculate f''(0)
# # make zero Tensor from x Tensor
zerox = x.cpu().clone()
zerox[0][0] = 0
f_xx_0 = d2y_dx2(mlp(zerox), zerox)[0]
print("f''(0) = {}".format(f_xx_0[0]))

# history 
# f''(0) = 0.3325398571314273 
# f''(0) = 0.3321312229973239

f''(0) = 1.0211181181093765


In [ ]:
#[TODO]
domain = x.detach().numpy().flatten()
# exact = (1 - x**2 / 6).detach().numpy().flatten()
predict = mlp.forward(x).detach().numpy().flatten()

plt.figure(figsize=(10, 5))
plt.xlabel('x')
plt.ylabel('y')

# plt.plot(domain, exact,'k--', markersize=.1, label='Exact')
plt.plot(domain, predict,'b.', markersize=1, label='Predict')
plt.legend()

# plt.savefig('exact-predict.eps', bbox_inches='tight', format='eps')

In [ ]:
plt.figure(figsize=(10, 5))

res= get_loss(x, ret_res=True)[1].detach().numpy()
plt.xlabel('x')
plt.ylabel('Residual')
plt.plot(domain, res)
plt.savefig('residual-loss.eps', bbox_inches='tight', format='eps')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.log(losses))
plt.xlabel('epoch')
plt.ylabel('log(loss)')

plt.savefig('loss.eps', bbox_inches='tight', format='eps')